In [4]:
# import cv2

# # Initialize the webcam
# cap = cv2.VideoCapture(0)

# # Check if the webcam is opened correctly
# if not cap.isOpened():
#     raise IOError("Cannot open webcam")

# # Read a frame from the webcam
# ret, frame = cap.read()

# # Release the webcam
# cap.release()

# # Save the captured image
# cv2.imwrite('test.jpg', frame)
import cv2
import time

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Warm-up: Read and display frames for 2 seconds
start_time = time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    cv2.imshow('Preview', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if time.time() - start_time > 5:
        break

# Capture the final frame
ret, frame = cap.read()
if ret:
    cv2.imwrite('test.jpg', frame)
    print("Image captured and saved as test.jpg")
else:
    print("Failed to capture image")

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Image captured and saved as test.jpg


In [15]:
!pip install numpy==1.24.4

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


In [5]:
import numpy as np
print(np.__version__)

1.24.4


In [ ]:
disease_types = ['Pepper__bell___Bacterial_spot','Pepper__bell___healthy','Potato___Early_blight','Potato___Late_blight','Potato___healthy','Tomato_Bacterial_spot','Tomato_Early_blight','Tomato_Late_blight','Tomato_Leaf_Mold','Tomato_Septoria_leaf_spot','Tomato_Spider_mites_Two_spotted_spider_mite','Tomato__Target_Spot','Tomato__Tomato_YellowLeaf__Curl_Virus','Tomato__Tomato_mosaic_virus','Tomato_healthy']

In [34]:
import numpy as np
from PIL import Image
import tflite_runtime.interpreter as tflite

# Load the TFLite model
interpreter = tflite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input & output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load image and preprocess
img = Image.open("test.jpg").resize((64, 64))  # Resize to match model
input_data = np.expand_dims(np.array(img, dtype=np.float32) / 255.0, axis=0)

# Set tensor and invoke
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get prediction
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(output_data)

print("Predicted class index:", predicted_class)


Predicted class index: 6


In [43]:
import numpy as np
from PIL import Image
import tflite_runtime.interpreter as tflite

# Define the disease types list
disease_types = [
    'Pepper__bell___Bacterial_spot',
    'Pepper__bell___healthy',
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy',
    'Tomato_Bacterial_spot',
    'Tomato_Early_blight',
    'Tomato_Late_blight',
    'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot',
    'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot',
    'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus',
    'Tomato_healthy'
]

# Load the TFLite model
interpreter = tflite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input & output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape expected by model:", input_details[0]['shape'])

# Load image and preprocess
img = Image.open("test.jpg").resize((64, 64))  # Resize to match model
input_data = np.expand_dims(np.array(img, dtype=np.float32) / 255.0, axis=0)

# Set tensor and invoke
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get prediction
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(output_data)
print("Output raw scores:", output_data)
print("Sum of scores (should be ~1.0 if softmax):", np.sum(output_data))
# print("Input dtype:", input_details[0]['dtype'])
# print("Output dtype:", output_details[0]['dtype'])

# Retrieve the disease label
predicted_label = disease_types[predicted_class]

print("Predicted class index:", predicted_class)
print("Predicted disease label:", predicted_label)

Input shape expected by model: [ 1 64 64  3]
Output raw scores: [[9.6489974e-05 6.2589941e-04 1.3634019e-05 3.8881024e-07 2.2641209e-06
  6.0926538e-07 9.9782640e-01 9.3772018e-04 6.9394206e-05 4.0118385e-04
  1.5544374e-06 8.6129758e-06 4.0829683e-08 1.5591728e-05 2.2331136e-07]]
Sum of scores (should be ~1.0 if softmax): 1.0
Predicted class index: 6
Predicted disease label: Tomato_Early_blight
